In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , classification_report
import re, string


In [2]:
data_fake = pd.read_csv('/content/drive/MyDrive/My Practice/Datasets/Fake.csv')
data_true = pd.read_csv('/content/drive/MyDrive/My Practice/Datasets/True.csv')


In [3]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
data_fake['class'] = 0
data_true['class'] = 1

In [6]:
data_fake.shape, data_true.shape

((23481, 5), (21417, 5))

In [7]:
data_fake_manual_testing =  data_fake.tail(10)
for i in range(23480,23470,-1):
  data_fake.drop([i],axis=0,inplace=True)

data_true_manual_testing =  data_true.tail(10)

for i in range(21416,21406,-1):
  data_true.drop([i],axis=0,inplace=True)

In [8]:
data_fake.shape, data_true.shape

((23471, 5), (21407, 5))

In [9]:
data_fake_manual_testing['class'] = 0
data_true_manual_testing['class'] = 1

<ipython-input-9-90008d39c97b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing['class'] = 0
<ipython-input-9-90008d39c97b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing['class'] = 1


In [10]:
data_merge = pd.concat([data_fake,data_true],axis=0)
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [11]:
data = data_merge.drop(['title','subject','date'],axis=1)

In [12]:
data.isnull().sum()

text     0
class    0
dtype: int64

In [13]:
data = data.sample(frac=1)

In [14]:
data.head()

,text,class
1312,"A new Politico/Morning Consult poll is out, an...",0
9021,WASHINGTON (Reuters) - The White House on Thur...,1
5354,MUNICH (Reuters) - Saudi Arabia’s foreign mini...,1
20184,Clock Boy Part II. When you peel back the rott...,0
571,MOSCOW (Reuters) - Russian President Vladimir ...,1


In [15]:
data.reset_index(inplace=True)
data.drop(['index'],axis=1,inplace=True)

In [16]:
data.columns

Index(['text', 'class'], dtype='object')

In [17]:
data.head()

,text,class
0,"A new Politico/Morning Consult poll is out, an...",0
1,WASHINGTON (Reuters) - The White House on Thur...,1
2,MUNICH (Reuters) - Saudi Arabia’s foreign mini...,1
3,Clock Boy Part II. When you peel back the rott...,0
4,MOSCOW (Reuters) - Russian President Vladimir ...,1


In [18]:
def wordopt(text):
  text = text.lower()
  text = re.sub('\[.?\]','',text)
  text = re.sub("\\W"," ",text)
  text = re.sub('https?://S+ |www\.\S+','',text)
  text = re.sub('<.*?>+','',text)
  text = re.sub('[%s]' % re.escape(string.punctuation),'',text)
  text = re.sub('\n','',text)
  text = re.sub('\w*\d\w','',text)
  return text

In [19]:
data['text'] = data['text'].apply(wordopt)

In [20]:
x = data['text']
y = data['class']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.25)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(X_train)
xv_test = vectorization.transform(X_test)

In [23]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [24]:
pred_lr = LR.predict(xv_test)

In [25]:
LR.score(xv_test,y_test)

0.9854723707664884

In [26]:
print(classification_report(y_test,pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5847
           1       0.98      0.99      0.98      5373

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [27]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(xv_train,y_train)

DecisionTreeClassifier()

In [28]:
pred_dt = DT.predict(xv_test)
DT.score(xv_test,y_test)

0.9959001782531194

In [29]:
print(classification_report(y_test,pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5847
           1       1.00      1.00      1.00      5373

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



In [32]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(random_state=0)
RF.fit(xv_train,y_train)

RandomForestClassifier(random_state=0)

In [33]:
pred_rf = RF.predict(xv_test)
RF.score(xv_test,y_test)

0.9872549019607844

In [34]:
print(classification_report(y_test,pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5847
           1       0.99      0.98      0.99      5373

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [43]:
def output_label(n):
  if n == 0:
    return "Fake News"
  elif n == 1 :
    return "Not A Fake News"

def manual_testing(news):
  testing_news = {'text':[news]}
  new_def_test = pd.DataFrame(testing_news)
  new_def_test['text'] = new_def_test['text'].apply(wordopt)
  new_x_test =  new_def_test['text']
  new_xv_test = vectorization.transform(new_x_test)
  pred_LR = LR.predict(new_xv_test)
  pred_DT = DT.predict(new_xv_test)
  pred_RFC= RF.predict(new_xv_test)

  return print("\n\nLR Prediction: {} \nDT Prediction: {} \nRFC Prediction {}".format(output_label(pred_LR[0]), output_label(pred_DT[0]),
                                                                                      output_label(pred_RFC[0]) ))


In [44]:
news = str(input())
manual_testing(news)

The following statementsÂ were posted to the verified Twitter accounts of U.S. President Donald Trump, @realDonaldTrump and @POTUS.  The opinions expressed are his own.Â Reuters has not edited the statements or confirmed their accuracy.  @realDonaldTrump : - Based on the fact that the very unfair and unpopular Individual Mandate has been terminated as part of our Tax Cut Bill, which essentially Repeals (over time) ObamaCare, the Democrats & Republicans will eventually come together and develop a great new HealthCare plan! [0658 EST] - WOW, @foxandfrlends â€œDossier is bogus. Clinton Campaign, DNC funded Dossier. FBI CANNOT (after all of this time) VERIFY CLAIMS IN DOSSIER OF RUSSIA/TRUMP COLLUSION. FBI TAINTED.â€ And they used this Crooked Hillary pile of garbage as the basis for going after the Trump Campaign! [0824 EST] - All signs are that business is looking really good for next year, only to be helped further by our Tax Cut Bill. Will be a great year for Companies and JOBS! Stock